# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [39]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    # YOUR CODE HERE
    #cleaning and filtering
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df = nhl_df[nhl_df['year']==2018]
    nhl_df['team'] = nhl_df['team'].str.replace("*","")
    cities['NHL'] = cities['NHL'].str.replace("[\(\[].*?[\)\]]", "")
    #cities['Metropolitan area'] = cities['Metropolitan area'].str.replace("-+", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nhl_df=nhl_df.drop([0 , 9, 18, 26])
    nhl_df = nhl_df[['team','W','L']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]

    #separating team and city names
    nhl_df['city'] = nhl_df['team'].str.split(' ').str[0]
    nhl_df['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Vegas':'Las Vegas', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'New':'New York City', 'Anaheim': 'Los Angeles', 'Arizona': 'Phoenix', 'Florida': 'Miami', 'Colorado': 'Denver', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Carolina': 'Raleigh'}, inplace=True, regex=True)
    #nhl_df.loc[14,'city'] = "New York"
    #nhl_df.loc[16,'city'] = "New York"
    #nhl_df.loc[17,'city'] = "New York"
    nhl_df['team'] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nhl_df['team'].replace({'Leafs': 'Maple Leafs', 'Wings':'Red Wings', 'Jackets':'Blue Jackets', 'Knights':'Golden Knights'}, inplace=True, regex=True)
    nhl_df['team'].replace({'Rangers': 'RangersIslandersDevils', 'Islanders':'RangersIslandersDevils', 'Devils':'RangersIslandersDevils', 'Kings':'KingsDucks', 'Ducks':'KingsDucks'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']
    
    #grouping the same cities by teams
    nhl_df['W']=np.float64(nhl_df['W'])
    nhl_df['L']=np.float64(nhl_df['L'])
    #nhl_df = nhl_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nhl_df,on='team')
    result['Ratio'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    
    
    #raise NotImplementedError()
    
    #population_by_region = [pd.to_numeric(result['Population (2016 est.)[8]'], errors='coerce')]
    #win_loss_by_region = [pd.to_numeric(result['Ratio'], errors='coerce')]
    
    population_by_region = result['Population (2016 est.)[8]']
    win_loss_by_region = result['Ratio']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

global cities
global mlb
global nba
global nfl

mlb_df=pd.read_csv("assets/mlb.csv")
nhl=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nhl_helper():
    #code
    #cleaning and filtering
    global nhl
    nhl = nhl[nhl['year']==2018]
    nhl['team'] = nhl['team'].str.replace("*","")
    cities['NHL'] = cities['NHL'].str.replace("[\(\[].*?[\)\]]", "")
    #cities['Metropolitan area'] = cities['Metropolitan area'].str.replace("-+", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nhl=nhl.drop([0 , 9, 18, 26])
    nhl = nhl[['team','W','L']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]

    #separating team and city names
    nhl['city'] = nhl['team'].str.split(' ').str[0]
    nhl['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Vegas':'Las Vegas', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'New':'New York City', 'Anaheim': 'Los Angeles', 'Arizona': 'Phoenix', 'Florida': 'Miami', 'Colorado': 'Denver', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Carolina': 'Raleigh'}, inplace=True, regex=True)
    #nhl_df.loc[14,'city'] = "New York"
    #nhl_df.loc[16,'city'] = "New York"
    #nhl_df.loc[17,'city'] = "New York"
    nhl['team'] = nhl.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nhl['team'].replace({'Leafs': 'Maple Leafs', 'Wings':'Red Wings', 'Jackets':'Blue Jackets', 'Knights':'Golden Knights'}, inplace=True, regex=True)
    nhl['team'].replace({'Rangers': 'RangersIslandersDevils', 'Islanders':'RangersIslandersDevils', 'Devils':'RangersIslandersDevils', 'Kings':'KingsDucks', 'Ducks':'KingsDucks'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']
    
    #grouping the same cities by teams
    nhl['W']=np.float64(nhl_df['W'])
    nhl['L']=np.float64(nhl_df['L'])
    nhl = nhl.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nhl,on='team')
    result['Ratio'] = result['W']/(result['W']+result['L'])
    nhl_copy = result.copy()
    return nhl_copy

def nba_helper():
    #code
    #cleaning and filtering
    nba_df = nba_df[nba_df['year']==2018]
    nba_df['team'] = nba_df['team'].str.replace("*","")
    nba_df['team'] = nba_df['team'].str.replace("[\(\[].*?[\)\]]","")
    cities['NBA'] = cities['NBA'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nba_df = nba_df[['team','W','L','W/L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]

    #separating team and city names
    nba_df['city'] = nba_df['team'].str.split(' ').str[0]
    nba_df['city'].replace({'San':'San Antonio', 'Los':'Los Angeles', 'New':'New York City', 'Brooklyn':'New York City', 'Golden':'San Francisco Bay Area', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Indiana':'Indianapolis', 'Utah':'Salt Lake City', 'Oklahoma': 'Oklahoma City'}, inplace=True, regex=True)
    nba_df['team'] = nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nba_df['team'].replace({'Blazers': 'Trail Blazers', 'Lakers':'LakersClippers', 'Clippers':'LakersClippers', 'Knicks':'KnicksNets', 'Nets':'KnicksNets'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping the same cities by teams
    nba_df['W']=np.float64(nba_df['W'])
    nba_df['L']=np.float64(nba_df['L'])
    nba_df = nba_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nba_df,on='team')
    nba_copy = result.copy()
    return nba_copy
    
def mlb_helper():
    #code
    #cleaning and filtering
    mlb_df = mlb_df[mlb_df['year']==2018]
    cities['MLB'] = cities['MLB'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    mlb_df = mlb_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]

    #separating team and city names
    mlb_df['city'] = mlb_df['team'].str.split(' ').str[0]
    mlb_df['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Los':'Los Angeles', 'New':'New York City', 'Kansas':'Kansas City', 'Texas':'Dallas', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Colorado':'Denver', 'Arizona': 'Phoenix'}, inplace=True, regex=True)
    mlb_df.loc[28,'city'] = "San Francisco Bay Area"
    mlb_df.loc[29,'city'] = "San Diego"
    mlb_df['team'] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    mlb_df.loc[0,'team'] = "Red Sox"
    mlb_df.loc[3,'team'] = "Blue Jays"
    mlb_df.loc[8,'team'] = "White Sox"
    mlb_df['team'].replace({'Yankees': 'YankeesMets', 'Mets':'YankeesMets', 'Dodgers':'DodgersAngels', 'Angels':'DodgersAngels', 'Giants':'GiantsAthletics', 'Athletics':'GiantsAthletics', 'Cubs': 'CubsWhite Sox', 'White Sox': 'CubsWhite Sox'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping cities 
    mlb_df['W']=np.float64(mlb_df['W'])
    mlb_df['L']=np.float64(mlb_df['L'])
    mlb_df = mlb_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(mlb_df,on='team')
    mlb_copy = result.copy()
    return mlb_copy
    
def nfl_helper():
    #code
    #cleaning and filtering
    nfl_df = nfl_df[nfl_df['year']==2018]
    nfl_df['team'] = nfl_df['team'].str.replace("*","")
    nfl_df['team'] = nfl_df['team'].str.replace('+', "")
    cities['NFL'] = cities['NFL'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nfl_df=nfl_df.drop([0 , 5, 10, 15, 20, 25, 30, 35])
    nfl_df = nfl_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]

    #separating team and city names
    nfl_df['city'] = nfl_df['team'].str.split(' ').str[0]
    nfl_df.loc[1,'city'] = "Boston"
    nfl_df.loc[31,'city'] = "New Orleans"
    nfl_df.loc[4,'city'] = "New York City"
    nfl_df.loc[24,'city'] = "New York City"
    nfl_df['city'].replace({'Tampa': 'Tampa Bay Area', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'Arizona': 'Phoenix', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Kansas':'Kansas City', 'Green':'Green Bay', 'Carolina':'Charlotte', 'Tennessee':'Nashville'}, inplace=True, regex=True)
    nfl_df['team'] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_df['team'].replace({'Giants': 'GiantsJets', 'Jets':'GiantsJets', 'Rams':'RamsChargers', 'Chargers':'RamsChargers', '49ers':'49ersRaiders', 'Raiders':'49ersRaiders'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']
    
    #grouping the same cities by teams
    nfl_df['W']=np.float64(nfl_df['W'])
    nfl_df['L']=np.float64(nfl_df['L'])
    nfl_df = nfl_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nfl_df,on='team')
    
    nfl_copy = result.copy()
    return nfl_copy

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [41]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    #cleaning and filtering
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_df = nba_df[nba_df['year']==2018]
    nba_df['team'] = nba_df['team'].str.replace("*","")
    nba_df['team'] = nba_df['team'].str.replace("[\(\[].*?[\)\]]","")
    cities['NBA'] = cities['NBA'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nba_df = nba_df[['team','W','L','W/L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]

    #separating team and city names
    nba_df['city'] = nba_df['team'].str.split(' ').str[0]
    nba_df['city'].replace({'San':'San Antonio', 'Los':'Los Angeles', 'New':'New York City', 'Brooklyn':'New York City', 'Golden':'San Francisco Bay Area', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Indiana':'Indianapolis', 'Utah':'Salt Lake City', 'Oklahoma': 'Oklahoma City'}, inplace=True, regex=True)
    nba_df['team'] = nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nba_df['team'].replace({'Blazers': 'Trail Blazers', 'Lakers':'LakersClippers', 'Clippers':'LakersClippers', 'Knicks':'KnicksNets', 'Nets':'KnicksNets'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping the same cities by teams
    nba_df['W']=np.float64(nba_df['W'])
    nba_df['L']=np.float64(nba_df['L'])
    #nba_df = nba_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nba_df,on='team')
    result['W/L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    #raise NotImplementedError()
    
    population_by_region = result['Population (2016 est.)[8]']
    win_loss_by_region = result['W/L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #cleaning and filtering
    mlb_df = mlb_df[mlb_df['year']==2018]
    cities['MLB'] = cities['MLB'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    mlb_df = mlb_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]

    #separating team and city names
    mlb_df['city'] = mlb_df['team'].str.split(' ').str[0]
    mlb_df['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Los':'Los Angeles', 'New':'New York City', 'Kansas':'Kansas City', 'Texas':'Dallas', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Colorado':'Denver', 'Arizona': 'Phoenix'}, inplace=True, regex=True)
    mlb_df.loc[28,'city'] = "San Francisco Bay Area"
    mlb_df.loc[29,'city'] = "San Diego"
    mlb_df['team'] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    mlb_df.loc[0,'team'] = "Red Sox"
    mlb_df.loc[3,'team'] = "Blue Jays"
    mlb_df.loc[8,'team'] = "White Sox"
    mlb_df['team'].replace({'Yankees': 'YankeesMets', 'Mets':'YankeesMets', 'Dodgers':'DodgersAngels', 'Angels':'DodgersAngels', 'Giants':'GiantsAthletics', 'Athletics':'GiantsAthletics', 'Cubs': 'CubsWhite Sox', 'White Sox': 'CubsWhite Sox'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping cities 
    mlb_df['W']=np.float64(mlb_df['W'])
    mlb_df['L']=np.float64(mlb_df['L'])
    #mlb_df = mlb_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(mlb_df,on='team')
    result['W-L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    #raise NotImplementedError()
    
    population_by_region = result['Population (2016 est.)[8]']
    win_loss_by_region = result['W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [47]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #cleaning and filtering
    nfl_df = nfl_df[nfl_df['year']==2018]
    nfl_df['team'] = nfl_df['team'].str.replace("*","")
    nfl_df['team'] = nfl_df['team'].str.replace('+', "")
    cities['NFL'] = cities['NFL'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nfl_df=nfl_df.drop([0 , 5, 10, 15, 20, 25, 30, 35])
    nfl_df = nfl_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]

    #separating team and city names
    nfl_df['city'] = nfl_df['team'].str.split(' ').str[0]
    nfl_df.loc[1,'city'] = "Boston"
    nfl_df.loc[31,'city'] = "New Orleans"
    nfl_df.loc[4,'city'] = "New York City"
    nfl_df.loc[24,'city'] = "New York City"
    nfl_df['city'].replace({'Tampa': 'Tampa Bay Area', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'Arizona': 'Phoenix', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Kansas':'Kansas City', 'Green':'Green Bay', 'Carolina':'Charlotte', 'Tennessee':'Nashville'}, inplace=True, regex=True)
    nfl_df['team'] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_df['team'].replace({'Giants': 'GiantsJets', 'Jets':'GiantsJets', 'Rams':'RamsChargers', 'Chargers':'RamsChargers', '49ers':'49ersRaiders', 'Raiders':'49ersRaiders'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping the same cities by teams
    nfl_df['W']=np.float64(nfl_df['W'])
    nfl_df['L']=np.float64(nfl_df['L'])
    #nfl_df = nfl_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nfl_df,on='team')
    result['W-L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    #raise NotImplementedError()
    
    population_by_region = result['Population (2016 est.)[8]']
    win_loss_by_region = result['W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [63]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_ind


def nhl_correlation(): 
    # YOUR CODE HERE
    #cleaning and filtering
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df = nhl_df[nhl_df['year']==2018]
    nhl_df['team'] = nhl_df['team'].str.replace("*","")
    cities['NHL'] = cities['NHL'].str.replace("[\(\[].*?[\)\]]", "")
    #cities['Metropolitan area'] = cities['Metropolitan area'].str.replace("-+", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nhl_df=nhl_df.drop([0 , 9, 18, 26])
    nhl_df = nhl_df[['team','W','L']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]

    #separating team and city names
    nhl_df['city'] = nhl_df['team'].str.split(' ').str[0]
    nhl_df['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Vegas':'Las Vegas', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'New':'New York City', 'Anaheim': 'Los Angeles', 'Arizona': 'Phoenix', 'Florida': 'Miami', 'Colorado': 'Denver', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Carolina': 'Raleigh'}, inplace=True, regex=True)
    #nhl_df.loc[14,'city'] = "New York"
    #nhl_df.loc[16,'city'] = "New York"
    #nhl_df.loc[17,'city'] = "New York"
    nhl_df['team'] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nhl_df['team'].replace({'Leafs': 'Maple Leafs', 'Wings':'Red Wings', 'Jackets':'Blue Jackets', 'Knights':'Golden Knights'}, inplace=True, regex=True)
    nhl_df['team'].replace({'Rangers': 'RangersIslandersDevils', 'Islanders':'RangersIslandersDevils', 'Devils':'RangersIslandersDevils', 'Kings':'KingsDucks', 'Ducks':'KingsDucks'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']
    
    #grouping the same cities by teams
    nhl_df['W']=np.float64(nhl_df['W'])
    nhl_df['L']=np.float64(nhl_df['L'])
    #nhl_df = nhl_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nhl_df,on='team')
    result['Ratio'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    return result
    
def nba_correlation():
    # YOUR CODE HERE
    #cleaning and filtering
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_df = nba_df[nba_df['year']==2018]
    nba_df['team'] = nba_df['team'].str.replace("*","")
    nba_df['team'] = nba_df['team'].str.replace("[\(\[].*?[\)\]]","")
    cities['NBA'] = cities['NBA'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nba_df = nba_df[['team','W','L','W/L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]

    #separating team and city names
    nba_df['city'] = nba_df['team'].str.split(' ').str[0]
    nba_df['city'].replace({'San':'San Antonio', 'Los':'Los Angeles', 'New':'New York City', 'Brooklyn':'New York City', 'Golden':'San Francisco Bay Area', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Indiana':'Indianapolis', 'Utah':'Salt Lake City', 'Oklahoma': 'Oklahoma City'}, inplace=True, regex=True)
    nba_df['team'] = nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nba_df['team'].replace({'Blazers': 'Trail Blazers', 'Lakers':'LakersClippers', 'Clippers':'LakersClippers', 'Knicks':'KnicksNets', 'Nets':'KnicksNets'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping the same cities by teams
    nba_df['W']=np.float64(nba_df['W'])
    nba_df['L']=np.float64(nba_df['L'])
    #nba_df = nba_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nba_df,on='team')
    result['W/L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    return result
    
def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #cleaning and filtering
    mlb_df = mlb_df[mlb_df['year']==2018]
    cities['MLB'] = cities['MLB'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    mlb_df = mlb_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]

    #separating team and city names
    mlb_df['city'] = mlb_df['team'].str.split(' ').str[0]
    mlb_df['city'].replace({'Tampa': 'Tampa Bay Area', 'St.':'St. Louis', 'Los':'Los Angeles', 'New':'New York City', 'Kansas':'Kansas City', 'Texas':'Dallas', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Colorado':'Denver', 'Arizona': 'Phoenix'}, inplace=True, regex=True)
    mlb_df.loc[28,'city'] = "San Francisco Bay Area"
    mlb_df.loc[29,'city'] = "San Diego"
    mlb_df['team'] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    mlb_df.loc[0,'team'] = "Red Sox"
    mlb_df.loc[3,'team'] = "Blue Jays"
    mlb_df.loc[8,'team'] = "White Sox"
    mlb_df['team'].replace({'Yankees': 'YankeesMets', 'Mets':'YankeesMets', 'Dodgers':'DodgersAngels', 'Angels':'DodgersAngels', 'Giants':'GiantsAthletics', 'Athletics':'GiantsAthletics', 'Cubs': 'CubsWhite Sox', 'White Sox': 'CubsWhite Sox'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping cities 
    mlb_df['W']=np.float64(mlb_df['W'])
    mlb_df['L']=np.float64(mlb_df['L'])
    #mlb_df = mlb_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(mlb_df,on='team')
    result['W-L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    return result
    
def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #cleaning and filtering
    nfl_df = nfl_df[nfl_df['year']==2018]
    nfl_df['team'] = nfl_df['team'].str.replace("*","")
    nfl_df['team'] = nfl_df['team'].str.replace('+', "")
    cities['NFL'] = cities['NFL'].str.replace("[\(\[].*?[\)\]]", "")
    cities['Metropolitan area'] = cities['Metropolitan area'].str.split('–').str[0]

    #more cleaning and filtering
    nfl_df=nfl_df.drop([0 , 5, 10, 15, 20, 25, 30, 35])
    nfl_df = nfl_df[['team','W','L', 'W-L%']]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]

    #separating team and city names
    nfl_df['city'] = nfl_df['team'].str.split(' ').str[0]
    nfl_df.loc[1,'city'] = "Boston"
    nfl_df.loc[31,'city'] = "New Orleans"
    nfl_df.loc[4,'city'] = "New York City"
    nfl_df.loc[24,'city'] = "New York City"
    nfl_df['city'].replace({'Tampa': 'Tampa Bay Area', 'San':'San Francisco Bay Area', 'Los':'Los Angeles', 'Arizona': 'Phoenix', 'Washington':'Washington, D.C.', 'Minnesota':'Minneapolis', 'Kansas':'Kansas City', 'Green':'Green Bay', 'Carolina':'Charlotte', 'Tennessee':'Nashville'}, inplace=True, regex=True)
    nfl_df['team'] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_df['team'].replace({'Giants': 'GiantsJets', 'Jets':'GiantsJets', 'Rams':'RamsChargers', 'Chargers':'RamsChargers', '49ers':'49ersRaiders', 'Raiders':'49ersRaiders'}, inplace=True, regex=True)
    cities.columns = ['city','Population (2016 est.)[8]','team']

    #grouping the same cities by teams
    nfl_df['W']=np.float64(nfl_df['W'])
    nfl_df['L']=np.float64(nfl_df['L'])
    #nfl_df = nfl_df.groupby(by='team', as_index=False)['W', 'L'].mean()

    #creating the final df
    result = cities.merge(nfl_df,on='team')
    result['W-L%'] = result['W']/(result['W']+result['L'])
    
    #the population was in int, which would cause an error
    result['Population (2016 est.)[8]']=np.float64(result['Population (2016 est.)[8]'])
    result = result.groupby('city_x').mean().reset_index()
    return result

def sports_team_performance():
    # YOUR CODE HERE
    nhl = nhl_correlation()
    nba = nba_correlation()
    mlb = mlb_correlation()
    nfl = nfl_correlation()
    #creating pairs to be used, I analyzed each one to not be repeated
    nhl_nba = nhl.merge(nba,on='city_x')
    nhl_mlb = nhl.merge(mlb,on='city_x')
    nhl_nfl = nhl.merge(nfl,on='city_x')
    
    nba_mlb = nba.merge(mlb,on='city_x')
    nba_nfl = nba.merge(nfl,on='city_x')
    
    mlb_nfl = mlb.merge(nfl,on='city_x')
    
    #consulting only the pvalues in the test
    nhl_nba = stats.ttest_rel(nhl_nba['Ratio'], nhl_nba['W/L%'])[1]
    nhl_mlb = stats.ttest_rel(nhl_mlb['Ratio'], nhl_mlb['W-L%'])[1]
    nhl_nfl = stats.ttest_rel(nhl_nfl['Ratio'], nhl_nfl['W-L%'])[1]
    nba_mlb = stats.ttest_rel(nba_mlb['W/L%'], nba_mlb['W-L%'])[1]
    nba_nfl = stats.ttest_rel(nba_nfl['W/L%'], nba_nfl['W-L%'])[1]
    mlb_nfl = stats.ttest_rel(mlb_nfl['W-L%_x'], mlb_nfl['W-L%_y'])[1]
    
    #transforming the data into a dataframe
    #data = [[np.nan, nba_nfl, nhl_nfl, mlb_nfl], [nba_nfl, np.nan, nhl_nba, nba_mlb], [nhl_nfl, nhl_nba, np.nan, nhl_mlb], [mlb_nfl, nba_mlb, nhl_mlb, np.nan]]
    #df = pd.DataFrame(data, columns = ['NFL', 'NBA', 'NHL', 'MLB'])
    
    data = {'NFL':[np.nan, nba_nfl, nhl_nfl, mlb_nfl],
            'NBA':[nba_nfl, np.nan, nhl_nba, nba_mlb],
           'NHL':[nhl_nfl, nhl_nba, np.nan, nhl_mlb],
           'MLB':[mlb_nfl, nba_mlb, nhl_mlb, np.nan]}
    df = pd.DataFrame(data, index =['NFL',
                                'NBA',
                                'NHL',
                                'MLB'])
    
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    p_values = pd.DataFrame(df)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values